# Data Migration
**Prerequisites**
- Access to a CDF Project.
- Know how to use a terminal, so you can run `pygen` from the command line to 
  generate the SDK.
- Knowledge of your the data and data models.

In [1]:
import warnings
warnings.filterwarnings('ignore')
# This is just to enable improting the generated SDK from the examples folder in the pygen repository
import sys
from tests.constants import REPO_ROOT
sys.path.append(str(REPO_ROOT / "examples" ))

## Introduction to Problem

In the development of a solution, it is common that you move from one data model to another. Typically, the physicaly storage of data, the containers, stays the same, or are extended, 
and only the views of a data model change. In this case, there is no need for a data migration and all you do is update the solution code for the new model. 

If you, however, need to change how the data is stored in the containers, you need to move the data from one model to the other. A common to do this is to use CDF Transformations, but 
`pygen` can also be used for this case.

#### Advantage of CDF Transformations
* Can handle large volume of data, millions of instances.
* You can write the transformations in the UI using SQL.


#### Advantages of using `pygen` 
* Edges are automatically created.
* Can handle medium volume of data, hundreds of thousands of instances.
* Support doing very custom migration (what ever you can do in Python code).

**Why** would you **chose `pygen`** over **CDF Transformations**? If you are more comfortable with `Python` than writing `SQL`, you have data models with a of one-to-many edges, the number of nodes and edges (instances) is in the order of hundreds of thousands.




## Use Case

The use case we use in this guide is moving from 

In [1]:
from cognite.pygen import load_cognite_client_from_toml

In [2]:
from cognite.pygen.utils.cdf import clean_model_interactive, clean_space

In [3]:
client = load_cognite_client_from_toml()

In [5]:
clean_model_interactive(client, remove_space=True)

1) IntegrationTestsImmutable
2) IntegrationTestsMutable
3) Plant_Space
4) cogShop
5) market
6) tutorial_apm_simple
7) workshop

 q) Quit


Select option:  7


No models found


In [5]:
clean_space(client, "workshop")

CogniteAPIError: Invalid property reference '[node, space]' in a edge context. Only edge properties can be referenced. | code: 400 | X-Request-ID: 182a567d-7f6e-9c9a-af61-ba1aa5b7b631

## Retrieving Data

## Transformation